In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(1064881791906100)
print(torch.initial_seed())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



def inverse_standardize(tensor, mean, std):
    return tensor * std + mean


# 反标准化

# DataLoader部分
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data =  pd.read_csv("data1ramp0.4dt.csv")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data.iloc[idx, :5].values  # 前5列作为输入
        y = self.data.iloc[idx, 5:12].values  # 后7列作为输出
        # 重塑 x 为 (1, 5)，表示1个时间步，每步5个特征
        x = x.reshape(1, -1)
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


class GRUModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=64, output_size=7, num_layers=2):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU层
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

        # 全连接层，用于生成输出
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 初始化隐藏状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # GRU前向传播
        out, _ = self.gru(x, h0)

        # 我们只需要最后一个时间步的输出
        out = self.fc(out[:, -1, :])
        return out


# 创建模型实例
model = GRUModel(input_size=5, hidden_size=64, output_size=7, num_layers=2).to(device)


# 打印模型结构
print(model)


# 数据加载
dataset = CustomDataset('data1ramp0.4dt.csv')
dataloader = DataLoader(dataset, batch_size=2, shuffle=False, drop_last=True)

# 数据划分
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# 使用slice而不是random_split
train_dataset = torch.utils.data.Subset(dataset, range(0, train_size))
val_dataset = torch.utils.data.Subset(dataset, range(train_size, train_size + val_size))
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=False, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, drop_last=True)

# Unit
# model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam
criterion = nn.MSELoss()

# 学习率动态调整
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)
num_epochs = 10
# Early stopping
n_epochs_stop = 10
epochs_no_improve = 0
min_val_loss = float('inf')

predicted_vals = []
actual_vals = []

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # 确保输入形状正确：(batch_size, sequence_length, input_size)
        # 在这里，sequence_length = 1
        if inputs.dim() == 2:
            inputs = inputs.unsqueeze(1)

        # 前向传播
        outputs = model(inputs)

        # 计算损失
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    # Validation loss,Maybe it is ok

    model.eval()
    with torch.no_grad():
        total_loss = 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            if inputs.dim() == 2:
                inputs = inputs.unsqueeze(1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
        average_loss = total_loss / len(val_loader)
        print(f'Test Loss: {average_loss:.4f}')

            # # 预测与实际，方便出图
            # predicted_vals = outputs_inv_std
            # actual_vals = t0_inv_std
            #
        # if epoch == 9:
        #     predicted_vals.extend(outputs_inv_std.cpu().numpy().tolist())
        #     actual_vals.extend(t0.cpu().numpy().tolist())

    # scheduler.step(val_loss)
    # # 早停条件
    # if val_loss < min_val_loss:
    #     epochs_no_improve = 0
    #     min_val_loss = val_loss
    # else:
    #     epochs_no_improve += 1
    # if epochs_no_improve == n_epochs_stop:
    #     print("Early stopping!")
    #     break
    # print(f"Epoch {epoch + 1}, Training Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}")

for i, data in enumerate(train_loader, 0):
    tm, t0 = data

    # 反标准化预测的输出
    outputs = model(tm)
    outputs_inv_std = inverse_standardize(outputs, dataset.mean[0], dataset.std[0])
    # t0_inv_std = inverse_standardize(t0, dataset.mean_t0, dataset.std_t0)

    predicted_vals.extend(outputs_inv_std.detach().cpu().numpy().tolist())
    actual_vals.extend(t0.cpu().detach().numpy().tolist())
plt.figure(figsize=(12, 6))
plt.plot(predicted_vals, label='Predicted')
plt.plot(actual_vals, label='Actual t0')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.title('Comparison of Predicted Results and Actual t0 Values')
plt.show()

# plt.figure(figsize=(12,6))
# plt.plot(predict.detach().cpu().numpy(), label='Predicted')
# plt.plot(actual_vals, label='Actual t0')
# plt.xlabel('Sample Index')
# plt.ylabel('Value')
# plt.legend()
# plt.title('Comparison of Predicted Results and Actual t0 Values')
# plt.show()


1064881791906100
GRUModel(
  (gru): GRU(5, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=7, bias=True)
)


C:\ProgramData\Anaconda3\envs\OMPRL\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/10], Step [100/806], Loss: 54310776285954048.0000
Epoch [1/10], Step [200/806], Loss: 54930677505720320.0000
Epoch [1/10], Step [300/806], Loss: 45267709759324160.0000
Epoch [1/10], Step [400/806], Loss: 18118867146506240.0000
Epoch [1/10], Step [500/806], Loss: 8324807871430656.0000
Epoch [1/10], Step [600/806], Loss: 8532853939765248.0000
Epoch [1/10], Step [700/806], Loss: 8548983152574464.0000
Epoch [1/10], Step [800/806], Loss: 8555928114692096.0000
Test Loss: 8558695354679134.0000
Epoch [2/10], Step [100/806], Loss: 54310763401052160.0000
Epoch [2/10], Step [200/806], Loss: 54930668915785728.0000
Epoch [2/10], Step [300/806], Loss: 45267696874422272.0000
Epoch [2/10], Step [400/806], Loss: 18118862851538944.0000
Epoch [2/10], Step [500/806], Loss: 8324805187076096.0000
Epoch [2/10], Step [600/806], Loss: 8532850181668864.0000
Epoch [2/10], Step [700/806], Loss: 8548979394478080.0000
Epoch [2/10], Step [800/806], Loss: 8555924356595712.0000
Test Loss: 8558692027142590.000

AttributeError: 'CustomDataset' object has no attribute 'mean'

In [2]:
x

NameError: name 'x' is not defined

In [3]:
inputs

tensor([[[1.0000e+01, 2.3450e+06, 4.2857e+00, 4.0000e-01, 3.3333e-04]],

        [[0.0000e+00, 2.3450e+06, 4.2857e+00, 4.0000e-01, 3.3333e-04]]])

In [4]:
inputs.shape

torch.Size([2, 1, 5])

In [5]:
labels

tensor([[2.0000e-02, 5.1772e+01, 6.2649e+00, 2.4479e+08, 1.9875e+01, 4.8652e+02,
         1.0709e+02],
        [2.0000e-02, 5.1617e+01, 6.2479e+00, 2.4479e+08, 1.9875e+01, 4.8652e+02,
         1.0709e+02]])

In [6]:
labels.shape

torch.Size([2, 7])

In [7]:
loss

tensor(8.5602e+15)